In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense

In [2]:
# Step 1: Define the 6-line text
text = """The movie was thrilling and kept me on edge!
I found the plot confusing and the pacing too slow.
The acting was phenomenal, especially the lead actor.
This film is a must-watch for sci-fi fans.
The ending was disappointing and felt rushed.
Overall, it's a mixed bag but worth a look."""

In [3]:
# Step 2: Preprocess the text
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
# Create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:

# Create predictors and label
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# define the models 

In [7]:
#RNN MODEL 
def create_rnn_model():
    model = Sequential([
        Embedding(total_words, 10, input_length=max_sequence_len-1),
        SimpleRNN(50, return_sequences=False),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
#LSTM MODEL 
def create_lstm_model():
    model = Sequential([
        Embedding(total_words, 10, input_length=max_sequence_len-1),
        LSTM(50, return_sequences=False),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
#GRU MODEL 
def create_gru_model():
    model = Sequential([
        Embedding(total_words, 10, input_length=max_sequence_len-1),
        GRU(50, return_sequences=False),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
# Step 4: Train models
models = {
    'RNN': create_rnn_model(),
    'LSTM': create_lstm_model(),
    'GRU': create_gru_model()
}

for name, model in models.items():
    print(f"\nTraining {name} model...")
    model.fit(X, y, epochs=100, verbose=0)
    print(f"{name} model training completed.")

C:\Users\VANSHIKA\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Training RNN model...
RNN model training completed.

Training LSTM model...
LSTM model training completed.

Training GRU model...
GRU model training completed.


In [11]:
# Step 5: Predict next word
seed_text = "The movie was"
next_words = 1

for name, model in models.items():
    print(f"\nPredicting with {name} model...")
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    predicted_word = [word for word, index in tokenizer.word_index.items() if index == predicted_word_index][0]
    print(f"Input: {seed_text}, Predicted next word: {predicted_word}")


Predicting with RNN model...
Input: The movie was, Predicted next word: thrilling

Predicting with LSTM model...
Input: The movie was, Predicted next word: was

Predicting with GRU model...
Input: The movie was, Predicted next word: was
